# ĐỒ ÁN CUỐI KỲ (LTKHDL - CQ2017/2)
### GVHD: TS. Trần Trung Kiên

Họ và tên sinh viên: Nguyễn Hữu Huân

MSSV: 1712466

------

## Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

---

## Lấy dữ liệu

Dữ liệu mà em sử dụng ở đây được lấy từ [dataset của Yelp trên Kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset). Yelp là một công ty công nghệ lớn ở Mỹ với sản phẩm nổi bật là Yelp website và Yelp app cho phép người dùng tìm kiếm, đánh giá, bình luận, đặt chỗ dịch vụ của các cở sở kinh doanh. 

Khi tải dataset này về đồng nghĩa với việc đã chấp nhận điều khoản sử dụng (một file pdf khá dài) được viết bởi Yelp dành riêng cho dataset này ví dụ như là chỉ được sử dụng cho mục đích học tập giáo dục,...

Vì dataset này khá lớn (tầm 10GB) và chủ đề mà em quan tâm ở đây chỉ là nhà hàng, món ăn nên em chỉ sử dụng 2 file json là: "yelp_academic_dataset_business.json" và "yelp_academic_dataset_review.json" trong số 5 file của dataset. File "yelp_academic_dataset_business.json" chứa thông tin của các cơ sở kinh doanh, còn file "yelp_academic_dataset_review.json" chứa thông tin đánh giá của người dùng đối với cơ sở kinh doanh đó.  

-----

## Khám phá dữ liệu

### Data Description

Phần mô tả dữ liệu này Yelp không cung cấp trên Kaggle mà để ở trang chủ của mình [ở đây](https://www.yelp.com/dataset/documentation/main). Em có dịch sơ lại như sau:

- "yelp_academic_dataset_review.json":
    - *business_id*: mã id của doanh nghiệp
    - *name*: tên doanh nghiệp
    - *address*: địa chỉ
    - *city*: thành phố
    - *state*: mã bang 
    - *postal_code*: mã bưu chính
    - *latitude*: vĩ độ
    - *longitude*: kinh độ
    - *stars*: sao trung bình
    - *review_count*: số lượng review
    - *is_open*: có còn hoạt động hay không
    - *attribute*: một số thông tin về doanh nghiệp (đậu xe như thế nào, thanh toán thẻ được không,..)
    - *categories*: danh sách các loại của doanh nghiệp
    - *hours*: giở mở cửa (Thứ Hai, Thứ Ba,...)
- "yelp_academic_dataset_review.json":
    - *user_id*: mã id người dùng
    - *text*: nội dung review
    - *date*: ngày review
    - *review_id*: mã id của review
    - *business_id*: mã doanh nghiệp được review
    - *funny*: độ hài hước của review này được vote
    - *cool*: độ "ngầu" của review này được vote
    - *useful*: độ có ích của review được vote
    - *stars*: số sao review 

### Đọc dữ liệu vào DataFrame

Khá may mắn là mặc dù các file này là json nhưng cấu trúc nó khá "ngay ngắn" và có thể đọc thành DataFrame bằng hàm `pd.read_json()`. Nhưng cũng khá xui vì file về reviews khá nặng (6GB!), ban đầu em cũng thử cố đọc file này bằng cách định nghĩa các cột trước khi đọc và chia thành đọc các 'chunk' nhỏ, tuy nhiên máy vẫn bị "đơ". Để tiết kiệm thời gian thì em đã lên Kaggle tải file ["reviews.csv"](https://www.kaggle.com/darshank2019/review) mà người khác đã convert trước đó.

In [12]:
business_df = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)

 Vì file csv này là của phiên bản reviews cách đây 1 năm nên dung lượng chỉ còn khoảng 4GB. Nhưng 4GB vẫn là con số khá lớn, cho nên sau khi đọc data description em đã bỏ đọc một số cột em cảm thấy sẽ không cần dùng tới (theo tiêu chí bài này của em): "text", "cool", "funny" để việc đọc được diễn ra "suông sẻ" và nhanh hơn.

In [8]:
reviews_df = pd.read_csv('data/yelp_academic_dataset_review.csv', usecols=['user_id','date','review_id','business_id','stars','useful'])

,user_id,date,review_id,business_id,useful,stars
0,b'hG7b0MtEbXx5QzbzE6C_VA',b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',6,1.0
1,b'yXQM5uF2jS6es16SJzNHfg',b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,5.0
2,b'n6-Gk65cPZL6Uz8qRm3NYw',b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',3,5.0
3,b'dacAIZ6fTM6mqwW5uxkskg',b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',0,5.0
4,b'ssoyf2_x0EQMed6fgHeMyQ',b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',7,1.0
...,...,...,...,...,...,...
6685895,b'Xu_YazIajfsc7SRj1wJ6Mg',b'2018-07-03 12:17:27',b'0pCaq1hqAViS2N6ldlp_sQ',b'RXBFk3tVBxiTf3uOt9KExQ',1,5.0
6685896,b'V6BjjQICDO4q7TT3ZhaPWw',b'2013-05-22 15:32:08',b'jPCXuSBbI_cPocgoNBhdkg',b'yA6dKNm_zl1ucZCnwW8ZCg',13,1.0
6685897,b'PFiIECX8wuvi7P-1mCvIjA',b'2018-11-13 21:15:20',b'zwzO3yPdGbdgGy3-XQ0SWg',b'a192hdM0_UVCYLwPJv1Qwg',0,5.0
6685898,b'wu5R5_N7q5iqCBh6NBX26Q',b'2018-08-13 18:09:16',b'o8Nc2BJhKJXM6tRHJDRuPA',b'kOo4ZY2UQAX4j312mzQ8mA',1,5.0


### Khám phá

Quan sát kích thước của 2 DataFrame

In [13]:
business_df.shape

(209393, 14)

In [14]:
reviews_df.shape

(6685900, 6)

Dữ liệu khá bự 😥 14 cột, 209393 dòng với `business_df`, mỗi dòng là một doanh nghiệp. 6 cột 6685900 dòng với `reviews_df`, mỗi dòng là một review.

Xem thêm về thông tin các kiểu dữ liệu của từng cột, về ý nghĩa của từng cột thì đã có ở phần **Data Description**.

🏪 Thông tin về DataFrame business_df

In [15]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


**Một số lưu ý:**
- Có thể thấy postal_code là object thì hơi lạ, nhưng sau khi tìm hiểu thì postal code của Mỹ có bao gồm cả chữ cái (khác VN)
- hours và attribute ở đây là object (cụ thể là string) nhưng được lưu theo format dict (hay đúng hơn là format json object)
- categories là object (str) được lưu theo kiểu liệt kê: A, B, C

In [24]:
business_df.postal_code.sample(3)

78469       44060
134499    M5J 2N3
46601       28105
Name: postal_code, dtype: object

In [23]:
business_df.hours.sample(3)

34071    {'Monday': '9:30-20:30', 'Tuesday': '9:30-20:3...
67334                                                 None
14918    {'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'...
Name: hours, dtype: object

In [25]:
business_df.attributes.sample(3)

59793     {'BusinessAcceptsCreditCards': 'True', 'Busine...
133772    {'BusinessAcceptsCreditCards': 'True', 'ByAppo...
55702     {'RestaurantsPriceRange2': '2', 'Caters': 'Tru...
Name: attributes, dtype: object

In [26]:
business_df.categories.sample(3)

120148    Food, Cafes, French, Bakeries, Restaurants, Bi...
10476     Active Life, Cycling Classes, Trainers, Yoga, ...
63893                  Beer, Wine & Spirits, Food, Shopping
Name: categories, dtype: object

**Vấn đề:**
- Các cột chứa giá trị thiếu bao gồm: attributes, categories, hours. 2 cái còn lại thì không sao nhưng categories thiếu thì hơi khó khăn trong việc nhận diện loại dịch vụ. Có khả năng là sẽ được drop những dòng thiếu cột này.

✍ Thông tin về DataFrame reviews_df

In [17]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6685900 entries, 0 to 6685899
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      object 
 1   date         object 
 2   review_id    object 
 3   business_id  object 
 4   useful       int64  
 5   stars        float64
dtypes: float64(1), int64(1), object(4)
memory usage: 306.1+ MB


**Vấn đề:**
- Cột date cần phải đổi trừ object sang datetime
- stars cũng cần đổi sang int, vì thực ra cơ chế rating của Yelp chỉ có từ 1-> 5 sao thôi, không lẻ.

In [27]:
reviews_df.stars.unique()

array([1., 5., 4., 3., 2.])

In [3]:
# 1 = open, 0 = closeda
business_df = business_df[business_df['is_open']==1]

In [4]:
business_df.shape

(168903, 14)

In [11]:
reviews_df.head()

,user_id,date,review_id,business_id,useful,stars
0,b'hG7b0MtEbXx5QzbzE6C_VA',b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',6,1.0
1,b'yXQM5uF2jS6es16SJzNHfg',b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,5.0
2,b'n6-Gk65cPZL6Uz8qRm3NYw',b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',3,5.0
3,b'dacAIZ6fTM6mqwW5uxkskg',b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',0,5.0
4,b'ssoyf2_x0EQMed6fgHeMyQ',b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',7,1.0


----

## Đưa ra các câu hỏi cần trả lời

------

## Tìm câu trả lời cho từng câu hỏi

-------

## Nhìn lại quá trình làm đồ án

---

## Tài liệu tham khảo